In [1]:
pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
pip install huggingface_hub

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [6]:
import pandas as pd

# Load your dataset from a CSV file using a different encoding (e.g., ISO-8859-1)
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/labeled_text_dataset.csv", encoding='ISO-8859-1')

# Show first few rows of the dataset
print(data.head())



                                          transcript    label
0  On television and in print. Journalists uncove...     news
1  Did you see the news this morning? Yes, I saw ...     news
2  It's important to remember our physical health...   health
3  Mum, what's for dinner tonight? I haven't deci...  cooking
4  Sports equipment. Cameron has two sons. Michae...   sports


In [ ]:
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from datasets import load_dataset, Dataset
# import torch

# # Load the tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# # Example preprocessing for a custom dataset
# def preprocess_function(examples):
#     return tokenizer(examples['transcript'], truncation=True, padding=True, max_length=512)

# # Convert your dataset into the format accepted by the transformers library
# train_dataset = Dataset.from_pandas(train_data)  # train_data is a pandas DataFrame with 'transcript' and 'label' columns
# valid_dataset = Dataset.from_pandas(valid_data)

# train_dataset = train_dataset.map(preprocess_function, batched=True)
# valid_dataset = valid_dataset.map(preprocess_function, batched=True)

# # Training arguments
# training_args = TrainingArguments(
#     output_dir='./results',          # Output directory for checkpoints
#     evaluation_strategy="epoch",     # Evaluation during training
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

# # Trainer for fine-tuning BERT
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=valid_dataset,
# )

# # Train the model
# trainer.train()


In [7]:
# Map labels to integers
label_map = {
    "cooking": 0,
    "sports": 1,
    "news": 2,
    "finance": 3,
    "health": 4,
}

# Apply this map to the DataFrame
data['label'] = data['label'].map(label_map)


In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data['transcript'], data['label'], test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)


In [9]:
from transformers import BertTokenizer

# Load the tokenizer (BERT or any other model you prefer)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the datasets
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = CustomDataset(train_encodings, train_labels.tolist())
val_dataset = CustomDataset(val_encodings, val_labels.tolist())
test_dataset = CustomDataset(test_encodings, test_labels.tolist())


In [11]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    evaluation_strategy="epoch",     # Evaluate after each epoch
    learning_rate=0.0001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.391449
2,No log,0.096179
3,No log,0.371666
4,No log,0.386309
5,No log,0.388037


TrainOutput(global_step=90, training_loss=0.3492945777045356, metrics={'train_runtime': 77.4479, 'train_samples_per_second': 9.167, 'train_steps_per_second': 1.162, 'total_flos': 186815558430720.0, 'train_loss': 0.3492945777045356, 'epoch': 5.0})

In [14]:
from huggingface_hub import HfApi

In [15]:
model.push_to_hub("Nidhilakhan-17/nl_text_classification_model")
tokenizer.push_to_hub("Nidhilakhan-17/nl_text_classification_model")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nidhilakhan-17/nl_text_classification_model/commit/3f05f753bd88597621c09771b76b64275af62880', commit_message='Upload tokenizer', commit_description='', oid='3f05f753bd88597621c09771b76b64275af62880', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
api = HfApi()
api.create_repo(repo_id="Nidhilakhan-17/nl_text_classification_model_repo")

RepoUrl('https://huggingface.co/Nidhilakhan-17/nl_text_classification_model_repo', endpoint='https://huggingface.co', repo_type='model', repo_id='Nidhilakhan-17/nl_text_classification_model_repo')

In [18]:
from transformers import pipeline

# Load the model and tokenizer from the Hugging Face Hub
model_name = "Nidhilakhan-17/nl_text_classification_model"

classifier = pipeline("text-classification", model=model_name)

# Inference
text = "Staying hydrated is essential for overall health, especially during hot weather. Water helps regulate body temperature, keeps your skin healthy, and supports digestion. Aim to drink at least 8 glasses of water a day. In addition to water, consuming fruits and vegetables with high water content, like cucumbers and watermelon, can also help you stay hydrated. Don't wait until you're thirsty—keep a water bottle handy throughout the day!"
result = classifier(text)

print(result)


config.json:   0%|          | 0.00/995 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_4', 'score': 0.9849238395690918}]


In [19]:
# Evaluate the model on the test dataset
trainer.evaluate(test_dataset)


{'eval_loss': 0.30132874846458435,
 'eval_runtime': 1.2318,
 'eval_samples_per_second': 32.474,
 'eval_steps_per_second': 4.059,
 'epoch': 5.0}

In [ ]:
import torch

def classify_transcript(transcript):
    # Determine the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the correct device
    model.to(device)

    # Tokenize and move inputs to the correct device
    inputs = tokenizer(transcript, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    # Get model output
    outputs = model(**inputs)

    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

    return predicted_label

# Example usage:
new_transcript = "Staying hydrated is essential for overall health, especially during hot weather. Water helps regulate body temperature, keeps your skin healthy, and supports digestion. Aim to drink at least 8 glasses of water a day. In addition to water, consuming fruits and vegetables with high water content, like cucumbers and watermelon, can also help you stay hydrated. Don't wait until you're thirsty—keep a water bottle handy throughout the day!"
label = classify_transcript(new_transcript)
print(f"Predicted label: {label}")


Predicted label: 4
